# <center> **Pengantar OpenMC** </center> 

**Dhiya Salma Salsabila/24923304**

### **Pendahuluan**
**OpenMC** mensimulasikan partikel berupa **neutron dan foton** yang **bergerak secara stokastik (acak)** melalui sebuah **model yang di definisikan secara arbitrer (bebas)**.

<p>

Dalam reaktor nuklir, **neutron menjadi sangat penting karena dapat menyebabkan reaksi fisi**. Dengan mengetahui perilaku neutron maka dapat **menentukan seberapa sering reaksi fisi terjadi**. Hal ini juga **sebanding dengan energi yang dihasilkan**. 

#### **Deterministrik**
Dalam metode ini diselesaikan dengan persamaan yang dapat **menggambarkan perilaku rata-rata partikel** atau biasa di sebut dengan **persamaan transport neutron**
<p>

Persamaan ini merupakan **persamaan berdimensi tujuh (tiga dimensi untuk ruang, tiga untuk kecepatan, dan satu untuk waktu)** yang sangat sulit diselesaikan secara langsung. Untuk semua kasus, kecuali masalah yang paling sederhana, diperlukan beberapa bentuk diskretisasi. 
<p>

**Diskretisasi dilakukan dengan membagi seluruh ruang menjadi bagian-bagian kecil yang homogen**, kemudian menyelesaikan persamaan pada bagian-bagian kecil tersebut. Setelah melakukan diskretisasi dan berbagai pendekatan lainnya, kita dapat menghasilkan bentuk persamaan yang sesuai untuk diselesaikan dengan komputer. **Di antaranya adalah metode ordinat diskret, metode karakteristik, difusi beda hingga, dan metode nodal (pemecahan diferensial parsial)**.
<p>

**Kelebihan:**

- Kecepatan solusi: Tergantung pada metode yang digunakan, solusi dapat ditentukan dengan sangat cepat.
- Solusi global: Solusi yang dihasilkan adalah solusi global, artinya kita mengetahui perilaku rata-rata di seluruh domain.
- Kepastian solusi: Setelah masalah mencapai konvergensi, solusinya diketahui dengan pasti.
<p>

**Kekurangan:**

- Kerumitan model kompleks: Jika modelnya rumit, diperlukan pembuatan mesh yang canggih dan kompleks.
- Persiapan penampang multi-grup: Diperlukan penampang multi-grup yang memerlukan pengetahuan solusi sebelumnya (a priori).

#### **Monte Carlo**
Dalam metode ini menentukan perilaku rata-rata berbagai kuantitas fisik dalam suatu sistem
<p>

**Kelebihan:**

- Tanpa pembuatan mesh: Tidak diperlukan pembuatan mesh untuk membangun geometri. Dengan menggunakan constructive solid geometry (CSG), dimungkinkan untuk membuat model kompleks dengan permukaan melengkung.
- Fleksibilitas dalam penampang energi: Metode Monte Carlo dapat digunakan baik dengan penampang energi kontinu maupun multi-grup.
- Kemudahan simulasi paralel: Menjalankan simulasi secara paralel secara konsep sangat sederhana.
<p>

**Kekurangan:**

- Biaya komputasi tinggi: Karena bergantung pada pengambilan sampel acak berulang, metode ini memerlukan komputasi yang sangat mahal.
- Tidak otomatis memberikan solusi global: Simulasi tidak secara otomatis memberikan solusi global di seluruh domain – Anda harus secara spesifik meminta kuantitas yang diinginkan.
- Ketidakpastian stokastik: Bahkan setelah masalah mencapai konvergensi, diperlukan simulasi banyak partikel untuk mengurangi ketidakpastian stokastik.

#### **Satuan dalam OpenMC**
- Panjang : centimeter
- Energi : electronvolt
- Waktu : second

## <Center> **Konfigurasi Data Nuklir** </Center>
OpenMC menggunakan format HDF5, namun jika kita memiliki data dalam format ACE yang telah di olah NJOY seperti yang biasa di pakai dalam MCNP atau Serpent maka data tersebut harus di konversi terlebih dahulu ke HDF5.

##### **Mengolah format ACE**

In [ ]:
import openmc
# Contoh
u235 = openmc.data.IncidentNeutron.from_ace('92235.710nc')
u235.export_to_hdf5('U235.h5')

Jika terdapat beberapa file dengan format ACE untuk nuklida yang sama tetapi suhu yang berbeda maka dapat digunakan 

In [ ]:
# Interaksi neutron, untuk simulasi interaksi neutron dengan bahan pada berbagai suhu
u235 = openmc.data.IncidentNeutron.from_ace('92235.710nc')
for suffix in [711, 712, 713, 714, 715, 716]:
    u235.add_temperature_from_ace('92235.{}nc'.format(suffix))
u235.export_to_hdf5('U235.h5')

# Hamburan termal, untuk simulasi interaksi neutron dengan bahan pada suhu rendah, di mana efek hamburan termal signifikan
light_water = openmc.data.ThermalScattering.from_ace('lwtr.20t')
for suffix in range(21, 28):
    light_water.add_temperature_from_ace('lwtr.{}t'.format(suffix))
light_water.export_to_hdf5('lwtr.h5')

Setelah semua file ACE sudah di ubah ke dalam HDF5, maka file harus di ekspor ke dalam XML

In [ ]:
library = openmc.data.DataLibrary()
library.register_file('U235.h5')
library.register_file('lwtr.h5')
...
library.export_to_xml()

#### **Mengolah dari ENDF**

In [ ]:
# Contoh penambahan Zr-92, temperatur default 293.6 K
zr92 = openmc.data.IncidentNeutron.from_njoy('n-040_Zr_092.endf')

# Contoh penambahan dengan beberapa temperatur
zr92 = openmc.data.IncidentNeutron.from_njoy(
    'n-040_Zr_092.endf', temperatures=[300., 600., 1000.])

#### **Photon Cross Section**

In [ ]:
# Dari ACE
u = openmc.data.IncidentPhoton.from_ace('92000.12p')
u.export_to_hdf5('U.h5')

# Dari ENDF
u = openmc.data.IncidentPhoton.from_endf('photoat-092_U_000.endf',
                                         'atom-092_U_000.endf')

## <Center> **Running OpenMC** </Center>
untuk menjalanlan OpenMC, hal utama yang harus dilakukan ialah membuat file XML yang menggambarkan model yang akan disimulasikan. Ada 3 file yang diperlukan :
- **materials.xml**, file ini berisi **material apa yang ada dalam model** dan dari apa material tersebut terdiri. Selain itu, file ini menunjukkan di mana OpenMC harus mencari perpustakaan penampang lintang.
- **geometry.xml**, file ini menjelaskan bagaimana material yang **didefinisikan dalam materials.xml menempati wilayah ruang**. Volume fisik didefinisikan menggunakan geometri padat konstruktif, dijelaskan secara rinci dalam Mendefinisikan Geometri.
- **settings.xml**, file ini menunjukkan **mode apa yang harus dijalankan OpenMC**, berapa banyak partikel yang akan disimulasikan, definisi sumber, dan berbagai opsi lainnya.
- **tallies.xml**, file ini menjelaskan **kuantitas fisik apa yang harus dihitung selama simulasi (fluks, laju reaksi, arus, dll.)**.
- **plots.xml**, file ini memberikan **spesifikasi untuk menghasilkan plot irisan atau voxel dari geometri**.

In [ ]:
# Create materials
materials = openmc.Materials()
...
materials.export_to_xml()

# Create geometry
geometry = openmc.Geometry()
...
geometry.export_to_xml()

# Assign simulation settings
settings = openmc.Settings()
...
settings.export_to_xml()

## <center> **Material** </center>


Material dalam OpenMC didefinisikan sebagai sekumpulan nuklida/elemen dengan densitas tertentu dan dibuat menggunakan `openmc.Material`. Setelah sebuah material dibuat, nuklida dapat ditambahkan dengan `Material.add_nuclide()` dan elemen dapat ditambahkan dengan `Material.add_element()`. 
> **Densitas ditentukan menggunakan fraksi atom atau fraksi massa**
- **Elemen, zat kimia murni** yang terdiri dari atom dengan jumlah proton yang sama dalam intinya. Dicirikan oleh nomor atom (Z), misalnya: Hidrogen (H, Z=1), Karbon (C, Z=6), Uranium (U, Z=92).
- **Nuklida, inti atom tertentu yang ditentukan oleh jumlah proton (Z) dan neutron (N)**. misalnya: Uranium-235 (²³⁵U): Z = 92, N = 143, A = 235

In [ ]:
mat = openmc.Material()
# material Gd-152 dengan 0,5 persen atom
mat.add_nuclide('Gd152', 0.5, 'ao')

Densitas yang ditentukan untuk setiap nuklida/elemen bersifat relatif dan akan dinormalisasi berdasarkan densitas total material. 

**Densitas total diatur menggunakan metode `Material.set_density()`**. 

Densitas dapat ditentukan dalam gram per sentimeter kubik (g/cm3), atom per barn-cm (atom/b-cm), atau kilogram per meter kubik (kg/m3).

In [ ]:
mat.set_density('g/cm3', 4.5)

Metode `Material.add_element()` bekerja dengan cara yang sama seperti `Material.add_nuclide()`, tetapi alih-alih menentukan satu isotop dari suatu elemen, Anda menentukan elemen itu sendiri.

In [ ]:
mat.add_element('C', 1.0)
#atau
mat.add_element('aluminium', 1.0)

OpenMC menyimpan data mengenai massa atom dan kelimpahan alami dari semua isotop yang diketahui, lalu menggunakan data ini untuk menentukan isotop mana yang harus ditambahkan ke material. Ketika material diekspor ke XML untuk digunakan oleh eksekusi OpenMC, elemen alami apa pun akan otomatis diubah menjadi isotop-isotop yang terjadi secara alami.

Metode `Material.add_element()` juga dapat digunakan untuk menambahkan uranium dengan tingkat pengayaan tertentu melalui argumen `enrichment`

In [ ]:
# uranium dengan pengayaan 3,2%
mat.add_element('U', 1.0, enrichment=3.2)

Pengayaan juga dapat dilakukan untuk elemen yang terdiri dari dua isotop alami (misalnya, Li atau B) dalam satuan persen atom. Untuk mengaktifkan ini, tambahkan argumen `enrichment_target` ke `Material.add_element()`.

In [ ]:
# Pengayaan B-10 hingga 30%
mat.add_element('B', 1.0, enrichment=30.0, enrichment_target='B10')

Untuk mengayakan suatu isotop dalam satuan persen massa (`wo%`), tambahkan argumen tambahan `enrichment_type`.

In [ ]:
# Pengayaan Li-6 hingga 15% berat
mat.add_element('Li', 1.0, enrichment=15.0, enrichment_target='Li6',
                enrichment_type='wo')

**Catatan**:
> Seringkali, pustaka penampang lintang tidak benar-benar memiliki semua isotop alami dari suatu elemen. Misalnya, dalam ENDF/B-VII.1, evaluasi penampang lintang tersedia untuk O-16 dan O-17 tetapi tidak untuk O-18. Jika OpenMC mengetahui pustaka penampang lintang yang akan Anda gunakan (melalui variabel lingkungan OPENMC_CROSS_SECTIONS), maka OpenMC akan mencoba hanya memasukkan isotop yang memiliki data penampang lintang dalam model. Dalam kasus oksigen pada ENDF/B-VII.1, kelimpahan O-18 akan digabungkan dengan O-16.

### **Thermal Scattering**

Jika terdapat material moderator seperti air atau grafit, maka harus ditetapkan data hamburan termal (yang disebut S(α,β)) menggunakan metode `Material.add_s_alpha_beta()`. Sebagai contoh, untuk memodelkan air ringan, Anda perlu menambahkan hidrogen dan oksigen ke dalam suatu bahan dan kemudian menetapkan data hamburan termal c_H_in_H2O.

In [ ]:
water = openmc.Material()
water.add_nuclide('H1', 2.0)
water.add_nuclide('O16', 1.0)
water.add_s_alpha_beta('c_H_in_H2O')
water.set_density('g/cm3', 1.0)

Selain itu ada metode tambahan untuk hamburan termal dengan menggunakan NCrystal. Dengan Material.from_ncrystal() dapat memasukkan data suhu, komposisi material, dan densitas.

In [ ]:
mat = openmc.Material.from_ncrystal('Al_sg225.ncmat;temp=300K')

**Catatan :**
> NCrystal hanya menangani interaksi neutron energi rendah. Interaksi lainnya disediakan oleh file ACE

#### **Penamaan Material**
**Nuklida**
- `SymA` di mana "A" adalah nomor massa (misalnya, Fe56)

**Elemen**
- `Sym0` (misalnya, Fe0 atau C0)

**Keadaan tereksitasi**
- `SymA_eN` (misalnya, V51_e1 untuk keadaan tereksitasi pertama Vanadium-51). Ini hanya digunakan dalam data peluruhan.

**Keadaan metastabil**
- `SymA_mN` (misalnya, Am242_m1 untuk keadaan tereksitasi pertama Americium-242).

**Senyawa**
- `c_String_Describing_Material` (misalnya, c_H_in_H2O). Digunakan untuk data hamburan termal.

### **Temperatur**

Dalam OpenMC, definisi material dipisahkan dari spesifikasi suhu. Jika tidak ada spesifikasi suhu sel atau material, suhu default global dapat diatur untuk diterapkan ke semua sel dan material. **Semua suhu harus diberikan dalam satuan Kelvin**

In [ ]:
hot_fuel = openmc.Material()
hot_fuel.temperature = 1200.0  # temperature in Kelvin

### **Material Baru**

Di OpenMC, dimungkinkan untuk **mencampur sejumlah material untuk membuat material baru** dengan komposisi nuclide dan kerapatan yang benar. Metode `Material.mix_materials()` menerima daftar material dan daftar fraksi pencampurannya. **Fraksi pencampuran dapat diberikan sebagai fraksi atom, fraksi berat, atau fraksi volume. Jenis fraksi dapat ditentukan dengan memberikan 'ao', 'wo', atau 'vo'**

In [ ]:
mox_material = openmc.Material()
# Fraksi 0.97 mewakili 97% bahan bakar dan 0.03 mewakili 3% plutonium oksida berdasarkan berat, yang didefinisikan menggunakan fraksi berat (dengan parameter 'wo').
mox_material.mix_materials([fuel, plutonium_oxide], [0.97, 0.03], 'wo')

**Catatan:**
>  Jika fraksi menggunakan fraksi atom atau fraksi berat, fraksi harus berjumlah satu. Jika fraksi kurang dari satu, maka sisa fraksi akan ditetapkan sebagai material void (kosong).

Material dengan data hamburan termal S(α,β) tidak dapat digunakan dalam `metode Material.mix_materials()`. Namun, data hamburan termal dapat ditambahkan ke material yang dibuat dengan `Material.mix_materials()`.

Ini berarti bahwa meskipun tidak dapat langsung mencampur material yang memiliki data S(α,β), tetapi bisa membuat material campuran terlebih dahulu menggunakan `mix_materials()` dan kemudian menambahkan data hamburan termal setelah material campuran tersebut dibuat.

In [ ]:
# Setelah membuat material campuran
mox_material = openmc.Material()
mox_material.mix_materials([fuel, plutonium_oxide], [0.97, 0.03], 'wo')

# Menambahkan data hamburan termal S(alpha, beta) ke material campuran
mox_material.add_s_alpha_beta('H_in_H2O', 300)  # Misalnya, data untuk hidrogen di air pada suhu 300 K

Dalam OpenMC **material harus dibuat dalam bentuk XML**, Kelas Materials berfungsi seperti daftar, sehingga material dapat ditambahkan dengan cara melewatkan daftar ke konstruktor, menggunakan metode seperti append(), atau melalui operator +=. Selanjutnya ekspor koleksi tersebut menggunakan metode `Materials.export_to_xml()`.

In [ ]:
# Membuat instansi Materials
materials = openmc.Materials()

# Menambahkan beberapa material ke koleksi
fuel_material = openmc.Material(1)
fuel_material.add_nuclide('U235', 1.0, 'wo')  # contoh penambahan isotop
materials.append(fuel_material)

moderator_material = openmc.Material(2)
moderator_material.add_nuclide('H1', 2.0, 'wo')  # contoh penambahan isotop
materials.append(moderator_material)

# Mengekspor koleksi material ke dalam file materials.xml
materials.export_to_xml('materials.xml')

# Atau
materials = openmc.Materials()
materials.append(water)
materials += [uo2, zircaloy]
materials.export_to_xml()

# Atau
materials = openmc.Materials([water, uo2, zircaloy])
materials.export_to_xml()

### **Cross Section**

OpenMC menggunakan file bernama cross_sections.xml untuk menunjukkan di mana data penampang silang (cross section) dapat ditemukan di sistem file. File ini memiliki peran yang sama seperti xsdir untuk MCNP atau xsdata untuk Serpent.

OpenMC untuk menggunakan file tersebut dengan mengatur Materials.cross_sections atau dengan mengatur variabel lingkungan OPENMC_CROSS_SECTIONS ke path dari file cross_sections.xml.

In [ ]:
# Menetapkan path ke file cross_sections.xml dalam kelas Materials
materials = openmc.Materials()
materials.cross_sections = '/path/to/cross_sections.xml'

Pada bagian .profile di WSL harus di tambahkan

`export OPENMC_CROSS_SECTIONS=/path/to/cross_sections.xml`

## <center> **Geometri** </center>